Implementation of Densely Connected Convolutional Networks (https://arxiv.org/pdf/1608.06993.pdf). The idea here is propagating the previously learned properties to  succeeding ones .We are training here for Cifar10 dataset which consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images. 

1.Used Data Augmentation to create different images having flips and random rotation

2.Created a custom call back function to save the weights after each epoch to the drive.

In [0]:
!pip install -U -q PyDrive

import os
import glob
import keras
import numpy as np
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, merge, Activation,Conv2D, BatchNormalization,Concatenate,GlobalAveragePooling2D,AveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint,TensorBoard,Callback
from google.colab import files,auth
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.client import GoogleCredentials
from googleapiclient.http import MediaFileUpload
from googleapiclient.discovery import build
import scipy

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
drive_service = build('drive', 'v3')
auth.authenticate_user()

#!rm -f *.hdf5

In [0]:
downloaded = drive.CreateFile({'id':'171s-hZzwrLAu04SPCVNI7b6NSaALD-Rp'}) # replace the id with id of file you want to access
downloaded.GetContentFile('weights-improvement_-32_32-32_monday11110.hdf5')


In [0]:
batch_size = 64
num_classes = 10
epochs = 50
l = 11
num_filter = 36
compression = 0.5
dropout_rate = 0

In [0]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

121806848/170498071 [====================>.........] - ETA: 16s

170500096/170498071 [==============================] - 54s 0us/step


In [0]:
x_resized=np.zeros(shape=(50000,24,24,3))
for i in range(len(x_train)):
  x_resized[i] = scipy.misc.imresize(x_train[i],(24,24,3))

/usr/local/lib/python3.6/dist-packages/scipy/misc/pilutil.py:482: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if issubdtype(ts, int):
/usr/local/lib/python3.6/dist-packages/scipy/misc/pilutil.py:485: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif issubdtype(type(size), float):


In [0]:
datagen = ImageDataGenerator(rotation_range=15,width_shift_range=0.1,height_shift_range=0.1,horizontal_flip=True)
datagen.fit(x_resized)

In [0]:
def add_denseblock(input, num_filter):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        temp = concat
    return temp

In [0]:
def add_transition(input, num_filter):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    return avg

In [0]:
def output_layer(input):
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = GlobalAveragePooling2D(data_format="channels_last")(relu)
    output = Dense(num_classes, activation='softmax')(AvgPooling)
    return output

In [0]:
input = Input(shape=(None, None, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter)
First_Transition = add_transition(First_Block, num_filter)

Second_Block = add_denseblock(First_Transition, num_filter)
Second_Transition = add_transition(Second_Block, num_filter)

Third_Block = add_denseblock(Second_Transition, num_filter)
Third_Transition = add_transition(Third_Block, num_filter)

Last_Block = add_denseblock(Third_Transition,  num_filter)
output = output_layer(Last_Block)

In [0]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 3 972         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 3 144         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, None, None, 3 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [0]:
model.compile(loss='categorical_crossentropy',optimizer="adam",metrics=['accuracy'])

In [0]:
filepath="weights-improvement_-resized-{epoch:02d}_monday11110.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='acc', verbose=1, save_best_only=True, mode='max',period=1)
tcall=TensorBoard(log_dir='./Graph1', histogram_freq=0,  write_graph=True, write_images=True)
tcall.set_model(model)

class My_Callback(Callback):
    def on_train_begin(self, logs={}):
        self.k=[]
        self.i=0
        self.s=[]
 
    def on_train_end(self, logs={}):
          files.download("Graph1")    
 
    def on_epoch_begin(self, logs={},*args):
        return
 
    def on_epoch_end(self, epoch, logs={},*args):
      try:
          self.k=sorted(glob.glob("*monday11110.hdf5"),key=os.path.getmtime)
          if not self.s:
            self.s.append(self.k[0])
          else:
            for i in range(len(self.k)):
              if self.k[i]!=self.s[self.i]:
                                    self.i+=1
                                    self.s.append(self.k[i]) 
          print("Saving the weight{0} in drive \n".format(self.s[-1]))
          file_metadata = {'name': self.s[-1],'mimeType': 'text/plain'}
          media = MediaFileUpload(self.s[-1], mimetype='text/plain')
          created = drive_service.files().create(body=file_metadata,media_body=media,fields='id').execute()
      except (BrokenPipeError, IOError):
              pass
    def on_batch_begin(self, batch, logs={}):
        return
 
    def on_batch_end(self, batch, logs={}):
           return

history=My_Callback()

In [0]:
model.fit_generator(datagen.flow(x_resized, y_train, batch_size=batch_size),
                    steps_per_epoch=len(x_resized)/batch_size, epochs=12,callbacks=[checkpoint,history,tcall])


Epoch 1/12
782/781 [==============================] - 197s 252ms/step - loss: 1.5242 - acc: 0.4378

Epoch 00001: acc improved from -inf to 0.43778, saving model to weights-improvement_-resized-01_monday11110.hdf5
Saving the weightweights-improvement_-resized-01_monday11110.hdf5 in drive 

Epoch 2/12
 71/781 [=>............................] - ETA: 2:53 - loss: 1.2490 - acc: 0.5482

782/781 [==============================] - 191s 244ms/step - loss: 1.1264 - acc: 0.5956

Epoch 00002: acc improved from 0.43778 to 0.59562, saving model to weights-improvement_-resized-02_monday11110.hdf5
Saving the weightweights-improvement_-resized-02_monday11110.hdf5 in drive 

Epoch 3/12
111/781 [===>..........................] - ETA: 2:42 - loss: 0.9971 - acc: 0.6471

782/781 [==============================] - 191s 244ms/step - loss: 0.9276 - acc: 0.6708

Epoch 00003: acc improved from 0.59562 to 0.67070, saving model to weights-improvement_-resized-03_monday11110.hdf5
Saving the weightweights-improvement_-resized-03_monday11110.hdf5 in drive 

Epoch 4/12
120/781 [===>..........................] - ETA: 2:40 - loss: 0.8453 - acc: 0.7039

782/781 [==============================] - 190s 243ms/step - loss: 0.8147 - acc: 0.7135

Epoch 00004: acc improved from 0.67070 to 0.71362, saving model to weights-improvement_-resized-04_monday11110.hdf5
Saving the weightweights-improvement_-resized-04_monday11110.hdf5 in drive 

Epoch 5/12
122/781 [===>..........................] - ETA: 2:40 - loss: 0.7405 - acc: 0.7433

782/781 [==============================] - 190s 243ms/step - loss: 0.7397 - acc: 0.7413

Epoch 00005: acc improved from 0.71362 to 0.74130, saving model to weights-improvement_-resized-05_monday11110.hdf5
Saving the weightweights-improvement_-resized-05_monday11110.hdf5 in drive 

Epoch 6/12
122/781 [===>..........................] - ETA: 2:40 - loss: 0.7133 - acc: 0.7494

...

782/781 [==============================] - 190s 243ms/step - loss: 0.5793 - acc: 0.8002

Epoch 00009: acc improved from 0.78900 to 0.80026, saving model to weights-improvement_-resized-09_monday11110.hdf5
Saving the weightweights-improvement_-resized-09_monday11110.hdf5 in drive 

Epoch 10/12
122/781 [===>..........................] - ETA: 2:41 - loss: 0.5360 - acc: 0.8110

782/781 [==============================] - 191s 244ms/step - loss: 0.5506 - acc: 0.8080

Epoch 00010: acc improved from 0.80026 to 0.80798, saving model to weights-improvement_-resized-10_monday11110.hdf5
Saving the weightweights-improvement_-resized-10_monday11110.hdf5 in drive 

Epoch 11/12
122/781 [===>..........................] - ETA: 2:39 - loss: 0.5085 - acc: 0.8245

782/781 [==============================] - 190s 243ms/step - loss: 0.5273 - acc: 0.8173

Epoch 00011: acc improved from 0.80798 to 0.81722, saving model to weights-improvement_-resized-11_monday11110.hdf5
Saving the weightweights-improvement_-resized-11_monday11110.hdf5 in drive 

Epoch 12/12
122/781 [===>..........................] - ETA: 2:41 - loss: 0.4832 - acc: 0.8312

782/781 [==============================] - 191s 244ms/step - loss: 0.5031 - acc: 0.8238

Epoch 00012: acc improved from 0.81722 to 0.82378, saving model to weights-improvement_-resized-12_monday11110.hdf5
Saving the weightweights-improvement_-resized-12_monday11110.hdf5 in drive 



MessageError: ignored

In [0]:
filepath="weights-improvement_-32*32-{epoch:02d}_tuesday11110.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='acc', verbose=1, save_best_only=True, mode='max',period=1)
tcall=TensorBoard(log_dir='./Graph2', histogram_freq=0,  write_graph=True, write_images=True)
tcall.set_model(model)

class My_Callback(Callback):
    def on_train_begin(self, logs={}):
        self.k=[]
        self.i=0
        self.s=[]
 
    def on_train_end(self, logs={},*args):
          return  
 
    def on_epoch_begin(self, logs={},*args):
        return
 
    def on_epoch_end(self, epoch, logs={},*args):
      try:
          self.k=sorted(glob.glob("*tuesday11110.hdf5"),key=os.path.getmtime)
          if not self.s:
            self.s.append(self.k[0])
          else:
            for i in range(len(self.k)):
              if self.k[i]!=self.s[self.i]:
                                    self.i+=1
                                    self.s.append(self.k[i]) 
          print("Saving the weight{0} in drive \n".format(self.s[-1]))
          file_metadata = {'name': self.s[-1],'mimeType': 'text/plain'}
          media = MediaFileUpload(self.s[-1], mimetype='text/plain')
          created = drive_service.files().create(body=file_metadata,media_body=media,fields='id').execute()
      except (BrokenPipeError, IOError):
              pass
    def on_batch_begin(self, batch, logs={}):
        return
 
    def on_batch_end(self, batch, logs={}):
           return

history=My_Callback()

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]
datagen = ImageDataGenerator(rotation_range=15,width_shift_range=0.1,height_shift_range=0.1,horizontal_flip=True)
datagen.fit(x_train)

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


In [0]:
model.load_weights("weights-improvement_-32_32-32_monday11110.hdf5")

In [0]:
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=len(x_train)/batch_size, epochs=4,callbacks=[checkpoint,history,tcall],
                   validation_data=(x_test, y_test))

Epoch 1/4
782/781 [==============================] - 307s 392ms/step - loss: 0.1730 - acc: 0.9391 - val_loss: 0.4227 - val_acc: 0.8766

Epoch 00001: acc improved from -inf to 0.93928, saving model to weights-improvement_-32*32-01_tuesday11110.hdf5
Saving the weightweights-improvement_-32*32-01_tuesday11110.hdf5 in drive 

Epoch 2/4
 67/781 [=>............................] - ETA: 4:14 - loss: 0.1516 - acc: 0.9487

782/781 [==============================] - 297s 380ms/step - loss: 0.1699 - acc: 0.9414 - val_loss: 0.4581 - val_acc: 0.8739

Epoch 00002: acc improved from 0.93928 to 0.94148, saving model to weights-improvement_-32*32-02_tuesday11110.hdf5
Saving the weightweights-improvement_-32*32-02_tuesday11110.hdf5 in drive 

Epoch 3/4
101/781 [==>...........................] - ETA: 4:01 - loss: 0.1601 - acc: 0.9425

782/781 [==============================] - 297s 380ms/step - loss: 0.1677 - acc: 0.9417 - val_loss: 0.3604 - val_acc: 0.8929

Epoch 00003: acc improved from 0.94148 to 0.94188, saving model to weights-improvement_-32*32-03_tuesday11110.hdf5
Saving the weightweights-improvement_-32*32-03_tuesday11110.hdf5 in drive 

Epoch 4/4
107/781 [===>..........................] - ETA: 4:00 - loss: 0.1579 - acc: 0.9448

782/781 [==============================] - 298s 381ms/step - loss: 0.1614 - acc: 0.9420 - val_loss: 0.3959 - val_acc: 0.8871

Epoch 00004: acc improved from 0.94188 to 0.94208, saving model to weights-improvement_-32*32-04_tuesday11110.hdf5
Saving the weightweights-improvement_-32*32-04_tuesday11110.hdf5 in drive 



In [0]:
model.evaluate(x_test,y_test)

10000/10000 [==============================] - 24s 2ms/step


[0.39590889992415906, 0.8871]